In [1]:
import os
DEVELOPMENT_MODE = False
# Detect if we're running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

# Install if in Colab
if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis
    # Install a faster Node version
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs  # noqa

# Hot reload in development mode & not running on the CD
if not IN_COLAB:
    from IPython import get_ipython
    ip = get_ipython()
    if not ip.extension_manager.loaded:
        ip.extension_manager.load('autoreload')
        %autoreload 2
        
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
# if IN_COLAB or not DEVELOPMENT_MODE:
#     pio.renderers.default = "colab"
# else:
#     pio.renderers.default = "notebook_connected"
pio.renderers.default = 'notebook_connected' #1回設定しておけばいい！
print(f"Using renderer: {pio.renderers.default}")

import circuitsvis as cv
# Testing that the library works
cv.examples.hello("keno")

# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial

# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

Using renderer: notebook_connected
[1721127914.297524] [b9bc9f0b61cd:837144:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_id = "meta-llama/Meta-Llama-3-8B"
# model_id = "microsoft/Phi-3-mini-128k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
# tf_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
model = HookedTransformer.from_pretrained_no_processing(model_id, torch_dtype=torch.bfloat16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model meta-llama/Meta-Llama-3-8B into HookedTransformer


In [3]:
tokenizer.encode(':')

[128000, 25]

In [4]:
zero_shot_prompt = """Write a high-quality answer for the given question using only the provided search results (some of which might be irrelevant).

Document [1](Title: Drafting of the Universal Declaration of Human Rights) P. C. Chang of Republic of China (Taiwan), and Hansa Mehta of India among others. While not a member of the drafting committee, the French philosopher Jacques Maritain was influential in the lead up to the drafting of the Universal Declaration, advocacy for it within UNESCO in 1947-8, and in its subsequent advancement. The Drafting Committee included John Peters Humphrey was newly appointed as Director of the Division of Human Rights within the United Nations Secretariat. In this role, he produced the first draft of a list of rights that were to form the basis of the Declaration. The underlying
Document [2](Title: John Peters Humphrey) John Peters Humphrey John Peters Humphrey, OC (April 30, 1905 – March 14, 1995) was a Canadian legal scholar, jurist, and human rights advocate. He is most famous as the author of the first draft of the Universal Declaration of Human Rights. Humphrey was born to Frank Humphrey and Nellie Peter on April 30, 1905, in Hampton, New Brunswick. His childhood was touched by tragedy as he lost both his parents to cancer and he also lost one of his arms in an accident while playing with fire. Humphrey attended a boarding school where he endured teasing from other students;
Document [3](Title: Drafting of the Universal Declaration of Human Rights) Drafting of the Universal Declaration of Human Rights The Universal Declaration of Human Rights was drafted from early 1947 to late 1948 by Drafting Committee the first United Nations Commission on Human Rights. Further discussion and amendments were made by the Commission on Human Rights, the Economic and Social Council and the General Assembly of the United Nations. Members of the Commission who contributed significantly to the creation of the Declaration included Canadian John Peters Humphrey of the United Nations Secretariat, Eleanor Roosevelt of the United States (who chaired the Drafting Committee), René Cassin of France, Charles Malik of Lebanon,
Document [4](Title: John Peters Humphrey) of the bench, which sits beside two tall stone plinths, one of which has several articles from the Universal Declaration of Human Rights carved into it in English, French and Maliseet. The memorial sits on the front lawn of the County Court House in the center of the town. John Peters Humphrey John Peters Humphrey, OC (April 30, 1905 – March 14, 1995) was a Canadian legal scholar, jurist, and human rights advocate. He is most famous as the author of the first draft of the Universal Declaration of Human Rights. Humphrey was born to Frank Humphrey and Nellie Peter
Document [5](Title: Cyrus Cylinder) The Cylinder's text has traditionally been seen by biblical scholars as corroborative evidence of Cyrus' policy of the repatriation of the Jewish people following their Babylonian captivity (an act that the Book of Ezra attributes to Cyrus), as the text refers to the restoration of cult sanctuaries and repatriation of deported peoples. This interpretation has been disputed, as the text identifies only Mesopotamian sanctuaries, and makes no mention of Jews, Jerusalem, or Judea. The Cylinder has also been referred to by Mohammad Reza Pahlavi, the last Shah of Iran as the first declaration of universal human rights, a view rejected by some historians as anachronistic and a misunderstanding of the Cylinder's generic nature as a typical statement made by a new monarch at the beginning of his reign. Neil MacGregor, Director of the British Museum, has stated that the cylinder was "the first attempt we know about running a society, a state with different nationalities and faiths—a new kind of statecraft." It was adopted as a national symbol of Iran by the Imperial State which put it on display in Tehran in 1971 to commemorate 2,500 year celebration of the Persian Empire. On October 14, the Mohammad Reza Shah's sister, Princess Ashraf Pahlavi, presented the United Nations Secretary General U Thant with a replica of the Cylinder. The princess asserted that "the heritage of Cyrus was the heritage of human understanding, tolerance, courage, compassion and, above all, human liberty".
Document [6](Title: Declaration of the Rights of Man and of the Citizen) version of the Declaration was discussed by the representatives on the basis of a 24 article draft proposed by , led by . The draft was later modified during the debates. A second and lengthier declaration, known as the Declaration of the Rights of Man and Citizen of 1793, was written in 1793 but never formally adopted. The concepts in the Declaration come from the philosophical and political duties of the Enlightenment, such as individualism, the social contract as theorized by the Genevan philosopher Rousseau, and the separation of powers espoused by the Baron de Montesquieu. As can be seen
Document [7](Title: Universal Declaration of Human Rights) committee included René Cassin of France, Charles Malik of Lebanon, and P. C. Chang of the Republic of China. Humphrey provided the initial draft that became the working text of the Commission. According to Allan Carlson, the Declaration's pro-family phrases were the result of the Christian Democratic movement's influence on Cassin and Malik. Once the Committee finished its work in May 1948, the draft was further discussed by the Commission on Human Rights, the Economic and Social Council, the Third Committee of the General Assembly before being put to vote in December 1948. During these discussions many amendments and propositions
Document [8](Title: Human rights in the United States) prisoners at Guantanamo Bay and black sites, and extrajudicial targeted killings (Disposition Matrix). Some observers give the U.S. high to fair marks on human rights, while others charge it with a persistent pattern of human rights violations. The first human rights organization in the Thirteen Colonies of British America, dedicated to the abolition of slavery, was formed by Anthony Benezet in 1775. A year later, the Declaration of Independence announced that the Thirteen Colonies regarded themselves as independent states, and no longer a part of the British Empire. The Declaration stated "that all men are created equal, that they are
Document [9](Title: John Peters Humphrey) Rights. After consulting with the executive group of the Commission, chaired by Eleanor Roosevelt, Professor Humphrey prepared the first preliminary draft of what was to become the Universal Declaration of Human Rights. On the night of December 10, 1948, the General Assembly unanimously adopted the Declaration, dubbed by Mrs. Eleanor Roosevelt as "the international Magna Carta of all humankind." Humphrey remained with the UN for 20 years. During this period, the oversaw the implementation of 67 international conventions and the constitutions of dozens of countries. He worked in areas including the freedom of the press, status of women, and racial
Document [10](Title: Universal Declaration of Human Rights) the Rights of the Child, the United Nations Convention Against Torture, and many more. The Declaration continues to be widely cited by governments, academics, advocates, and constitutional courts, and by individuals who appeal to its principles for the protection of their recognised human rights. The Universal Declaration has received praise from a number of notable people. The Lebanese philosopher and diplomat Charles Malik called it "an international document of the first order of importance", while Eleanor Roosevelt—first chairwoman of the Commission on Human Rights (CHR) that drafted the Declaration—stated that it "may well become the international Magna Carta of all

Question: who wrote the first declaration of human rights
Answer →"""

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

input_ids = tokenizer(zero_shot_prompt, return_tensors="pt").input_ids.to('cuda')
outputs = model.generate(
    input_ids,
    max_new_tokens=32,
    eos_token_id=terminators,
    do_sample=False,
    temperature=None,
    top_p=None,
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))
del outputs


few_shot_prompt = """Write a high-quality answer for the given question using only the provided search results (some of which might be irrelevant).

Document [1](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was
Document [2](Title: Norwegian Americans) science, Ernest Lawrence won the Nobel Prize in Physics in 1939. Lars Onsager won the 1968 Nobel Prize in Chemistry. Norman Borlaug, father of the Green Revolution, won the Nobel Peace Prize in 1970. Christian B. Anfinsen won the Nobel Prize for chemistry in 1972. Ivar Giaever won the Nobel Prize in Physics 1973. Carl Richard Hagen is noted for his work in physics. In engineering, Clayton Jacobson II is credited with the invention of the modern personal watercraft. Ole Singstad was a pioneer of underwater tunnels. Ole Evinrude invented the first outboard motor with practical commercial application, recognizable today
Document [3](Title: Nobel Prize in Physics) Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was awarded to physicist Wilhelm Röntgen in recognition of the extraordinary services he
Document [4](Title: École normale supérieure (Paris)) was also awarded the Abel prize. In addition, eight "normaliens" have gone on to receive the Nobel Prize in Physics: Claude Cohen-Tannoudji, Pierre-Gilles de Gennes, Albert Fert, Alfred Kastler, Gabriel Lippmann, Louis Néel, Jean Baptiste Perrin and Serge Haroche, while other ENS physicists include such major figures as Paul Langevin, famous for developing Langevin dynamics and the Langevin equation. Alumnus Paul Sabatier won the Nobel Prize in Chemistry. A ranking of universities worldwide based on ratios of alumni to Nobel prize-winners published in 2016 by American scholars Stephen Hsu and Jonathan Wai placed ENS as the first university worldwide, far
Document [5](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice—in 1956 and 1972. Maria Skłodowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded
Document [6](Title: Nobel Prize in Physics) rendered by the discovery of the remarkable rays (or x-rays). This award is administered by the Nobel Foundation and widely regarded as the most prestigious award that a scientist can receive in physics. It is presented in Stockholm at an annual ceremony on 10 December, the anniversary of Nobel's death. Through 2018, a total of 209 individuals have been awarded the prize. Only three women (1.4% of laureates) have won the Nobel Prize in Physics: Marie Curie in 1903, Maria Goeppert Mayer in 1963, and Donna Strickland in 2018. Alfred Nobel, in his last will and testament, stated that his
Document [7](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the "Hindustan Times", "The 2005 Nobel prize for Physics was awarded for my work, but I wasn't the one to get it. Each one of the discoveries that this Nobel was given for work based on my research." Sudarshan also commented on not being selected for the 1979 Nobel, "Steven Weinberg, Sheldon Glashow and Abdus Salam built on work I had done as a 26-year-old student. If you give a prize for a building, shouldn’t the fellow who built the first floor be given the prize before those who built the second
Document [8](Title: Svante Arrhenius) Wilhelm Ostwald, Theodore Richards) and to attempt to deny them to his enemies (Paul Ehrlich, Walther Nernst, Dmitri Mendeleev). In 1901 Arrhenius was elected to the Swedish Academy of Sciences, against strong opposition. In 1903 he became the first Swede to be awarded the Nobel Prize in chemistry. In 1905, upon the founding of the Nobel Institute for Physical Research at Stockholm, he was appointed rector of the institute, the position where he remained until retirement in 1927. He was elected a Foreign Member of the Royal Society (ForMemRS) in 1910. In 1911 he won the first Willard Gibbs Award.
Document [9](Title: Scientist) and pursued through a unique method, was essentially in place. Ramón y Cajal won the Nobel Prize in 1906 for his remarkable observations in neuroanatomy. Marie Curie became the first female to win the Nobel Prize and the first person to win it twice. Her efforts led to the development of nuclear energy and Radio therapy for the treatment of cancer. In 1922, she was appointed a member of the International Commission on Intellectual Co-operation by the Council of the League of Nations. She campaigned for scientist's right to patent their discoveries and inventions. She also campaigned for free access
Document [10](Title: Nobel Prize in Chemistry) on December 10, the anniversary of Nobel's death. The first Nobel Prize in Chemistry was awarded in 1901 to Jacobus Henricus van 't Hoff, of the Netherlands, "for his discovery of the laws of chemical dynamics and osmotic pressure in solutions." From 1901 to 2018, the award has been bestowed on a total of 180 individuals. Alfred Nobel stipulated in his last will and testament that his money be used to create a series of prizes for those who confer the "greatest benefit on mankind" in physics, chemistry, peace, physiology or medicine, and literature. Though Nobel wrote several wills during

Question: who got the first nobel prize in physics
Answer → Wilhelm Conrad Röntgen

Document [1](Title: Deadpool (video game)) 2015 in North America and Australia, and November 20, 2015 in the United Kingdom. It is unknown how Activision reacquired the rights to publish "Deadpool: The Video Game", but it is believed that the re-release was timed to help promote the then-upcoming Deadpool feature film, released roughly three months after the next-gen version of the game. High Moon Studios showed an early demo of the video game to journalists at Gamescom 2012, where the game earned a nomination for Best of Show. Several sites wrote positive impression of the demo that was shown at Gamescom and the Electronic Entertainment Expo
Document [2](Title: Deadpool (film)) footage was leaked online in July 2014, and was met with an overwhelmingly enthusiastic response. That September, Fox gave "Deadpool" a release date of February 12, 2016. Production was set to begin in March 2015, with Simon Kinberg joining as producer. Reynolds attributed Fox's green-lighting of the film entirely to the leak. He, Miller and the writers had previously discussed leaking the footage themselves, and Reynolds initially thought that Miller had done so. He later believed the leak came from someone at Fox. In exchange for being able to make the film the way they wanted, Fox gave the crew
Document [3](Title: Deadpool (film)) May of that year. Several vendors provided visual effects for the film, ranging from the addition of blood and gore to the creation of the CG character Colossus. "Deadpool" was released in the United States on February 12, 2016, after an unconventional marketing campaign. The film achieved both financial and critical success. It earned over $783 million against a $58 million budget, breaking numerous records: it became the highest-grossing R-rated film, the highest-grossing "X-Men" film, and the ninth-highest-grossing 2016 film. Critics praised Reynolds' performance, the film's style and faithfulness to the comics, and its action sequences. Some detractors criticized the
Document [4](Title: Paul Wernick) Tim Miller. The R-rated film was released on February 12, 2016. Its opening weekend was the highest grossing of all time for R-Rated films, and it went on to gross more than $780 million worldwide. The team's next project, sci-fi thriller "Life", was released in March 2017. The film was directed by Daniel Espinosa, produced by Skydance, and through Sony Pictures, with Ryan Reynolds, Jake Gyllenhaal, and Rebecca Ferguson starring. They wrote "Deadpool 2" in 2018, which like its predecessor was released to positive reviews. Wernick and Reese were announced as the co-screenwriters of the feature film adaptation of "Cowboy
Document [5](Title: Deadpool 2) Deadpool 2 is scheduled to be released in the United States on May 18, 2018.  A sequel, Deadpool 3, is in development.
Document [6](Title: Deadpool 2) film, explaining its general plot, was released at the end of March. "Forbes" contributor Scott Mendelson called it "pretty funny and mostly entertaining", but was disappointed in it being a "conventional" trailer compared to the more out-there videos previously released for the film. He explained that he thought the first film "had a winning lead character and fine character-centric jokes, but a pretty generic origin story plot that eventually became the thing it was critiquing", and was concerned that the sequel would turn out to be "a more standard 'superhero sequel' sell". Mendelson also noted the inclusion of T.J. Miller
Document [7](Title: Deadpool 2) at the convention moderated by Soni and featuring other cast members. Some of the scenes cut from the theatrical version of the film were debuted at the panel. At the end of September 2018, Fox announced that it would release an untitled "Deadpool" film in theaters on December 21. The studio suggested that press and fans "guess away" as to what this new film would be, but it was believed to be a re-cut version of "Deadpool 2" that would carry a PG-13 rating rather than being R-rated like the initial theatrical release. Reynolds hinted that the version of the
Document [8](Title: The New Mutants (film)) Bear would not be the main antagonist of the film, just that it was "very much inspired" by the run of the "New Mutants" comics in which Demon Bear was featured as the main villain. In January 2018, the film's release date was pushed back to February 22, 2019, which allowed it to avoid "Deadpool 2", which also had its release changed at that time to a date that would have had both films in theaters at the same time in certain markets, while also allowing time for the reshoots required to make the film more frightening. When asked about
Document [9](Title: Deadpool (film)) said that the company would be willing to make future R-rated Marvel films like "Deadpool", potentially under a "Marvel-R" brand, "as long as we let the audiences know what's coming". Before "Deadpool"s release, Fox green-lit a sequel with Reese and Wernick returning to write the screenplay. The involvement of Reynolds and Tim Miller was confirmed at the 2016 CinemaCon in April, but at the end of October, Miller left the film over "mutual creative differences" with Reynolds. The next month, David Leitch signed on to replace Miller for the sequel. Leitch first made a short film, "No Good Deed", which
Document [10](Title: Deadpool (film)) Screen Rant called it possibly "the best film marketing campaign in the history of cinema". HostGator's Jeremy Jensen attributed the campaign's success to Reynolds, and to Fox for embracing the film's R rating. "Deadpool"s world premiere was held at the Grand Rex in Paris on February 8, 2016, before its initial theatrical release in Hong Kong the next day. This was followed by releases in 49 other markets over the next few days, including the United States on February 12. The movie was released in several formats, including IMAX, DLP, premium large formats, and D-Box. Kinberg explained that unlike the

Question: when is the next deadpool movie being released
Answer → May 18, 2018

Document [1](Title: Geography of Nigeria) south atlantic ocean, locally known as the south western wind, or by its main name, The Tropical Maritime (MT) airmass. These two major wind systems in Nigeria are known as the trade winds. The tropical maritime airmass (MT) is responsible for Nigeria's rainy season. This wind (the tropical maritime airmass) invades the country from February in the southern part of Nigeria while it takes longer for the wind to fully cover the whole of the country, reaching the northern part of Nigeria in June. Its invasion is as a result of the northward retreat, of the tropical continental airmass (CT)
Document [2](Title: Geography of Nigeria) of low pressure to develop over west Africa and Nigeria (between March to May). The Tropical continental airmass (CT) from the Sahara Desert in the northern part of West Africa, is weakened due to the overheating of the land surface in west Africa and Nigeria at this time. The Tropical continental airmass (CT) begins to retreat northwards to the Sahara Desert due to massive heating of the land which transfers heat in the form of convection into the Tropical continental airmass (CT) which constitutes the main layer of air above the land. This transfer of heat in the Tropical continental
Document [3](Title: Geography of Nigeria) airmass (CT) in turn, causes the wind to expand and become lighter as this is the normal behaviour for winds moving above intensely heated grounds. The Tropical continental airmass (CT) loses its strength as a major airmass in the region of west Africa and over Nigeria at this time (around February in the southern part of Nigeria to June in northern Nigeria) and begins to retreat coupled with the rising of air in form of convection within this airmass (Tropical continental airmass (CT)), further weakening the dominance of the wind over west Africa and Nigeria. The Tropical continental airmass (CT)
Document [4](Title: Geography of Nigeria) northern end is south of the 15 degrees line at about 14 degrees. Nigeria's location in the wetter part of the easterly waves south of the 15 degree line creates wetter climatic conditions for Nigeria especially during the monsoons. The Tropical Continental Airmass (CT) locally known as the harmattan, is a wind originating from North Africa which crosses the Sahara Desert into west Africa to Nigeria. This airmass dominates Nigeria's climate during the dry season from December to March. The Tropical continental airmass is dusty and creates a haze within the atmosphere of west Africa and Nigeria when it predominates.
Document [5](Title: Geography of Nigeria) With the Intertropical Convergence Zone (ITCZ) swinging northward over West Africa from the Southern Hemisphere in April, heavy showers coming from pre-monsoonal convective clouds mainly in the form of squall lines also known as the north easterlies formed mainly as a result of the interactions of the two dominant airmasses in Nigeria known as the Maritime tropical(south westerlies) and the Continental tropical(north easterlies), begins in central Nigeria while the Monsoons from the south atlantic ocean arrives in central Nigeria in July bringing with it high humidity, heavy cloud cover and heavy rainfall which can be daily occurrence lasting till September when the monsoons gradually begin retreating southward to the southern part of Nigeria.
Document [6](Title: Awka) dry winds from across the Sahara desert. The monsoon winds from the Atlantic creates six months of heavy tropical rains, which occur between April and July, followed by a short dry period in August lasting two to three weeks with the rain resuming in September and October. This is followed by five months of dryness (November - March) marked by a Harmattan wind, also known as Ugulu in Igbo, which is a particularly dry and dusty wind which enters Nigeria in late December or in the early part of January and is characterized by a grey haze limiting visibility and
Document [7](Title: Geography of Nigeria) experiences inward blowing winds because winds are moving air blowing outwards from regions of high pressure to regions of low pressure. The Tropical Maritime Airmass is a warm humid and unstable trade wind due to its warmth. Convectional currents are easily set up within the airmass whenever there is little instability in the airmass as a result of a slight to a very high orographic uplift in mountainous regions like the obudu plateau or the heating of the land which can trigger the formation of cumulonimbus cloud leading to thunderstorms within the airmass. During the dominance of the Tropical Maritime
Document [8](Title: Geography of Nigeria) finally retreats from most part of Nigeria, and the West African atmosphere around April to May, leaving an empty atmosphere over Nigeria. The sun's rays enters into the atmosphere of Nigeria more intense than it does during the presence of the Tropical continental airmass, which contained dust (in form of haze) that reduced the intensity of the sun. The overheating of the west Africa land mass and Nigeria in particular creates a low pressure region over west Africa and Nigeria. This low pressure zone attracts the Tropical Maritime Airmass (MT) from the south Atlantic Ocean since areas of low pressures
Document [9](Title: Western Asia) two wind phenomena in Western Asia: the "sharqi" and the "shamal". The "sharqi" (or "sharki") is a wind that comes from the south and southeast. It is seasonal, lasting from April to early June, and comes again between late September and November. The winds are dry and dusty, with occasional gusts up to 80 kilometres per hour (50 miles per hour) and often kick up violent sand and dust storms that can carry sand a few thousand meters high, and can close down airports for short periods of time. These winds can last for a full day at the beginning
Document [10](Title: Onikwu) The dry season is accompanied by a dust laden airmass from the Sahara Desert, locally known as Harmattan, or by its main name, The Tropical Continental (CT) airmass, while the rainy season is heavily influenced by an airmass originating from the south Atlantic Ocean, locally known as the south west wind, or by its main name, The Tropical Maritime (MT) airmass. These two major wind systems in Nigeria are known as the trade winds. The region Onikwu/Ndoni is flood prone communities, this is because the inland part of Rivers state consists of tropical rainforest; towards the coast the typical Niger

Question: the south west wind blows across nigeria between
Answer → till September

Document [1](Title: Drafting of the Universal Declaration of Human Rights) P. C. Chang of Republic of China (Taiwan), and Hansa Mehta of India among others. While not a member of the drafting committee, the French philosopher Jacques Maritain was influential in the lead up to the drafting of the Universal Declaration, advocacy for it within UNESCO in 1947-8, and in its subsequent advancement. The Drafting Committee included John Peters Humphrey was newly appointed as Director of the Division of Human Rights within the United Nations Secretariat. In this role, he produced the first draft of a list of rights that were to form the basis of the Declaration. The underlying
Document [2](Title: John Peters Humphrey) John Peters Humphrey John Peters Humphrey, OC (April 30, 1905 – March 14, 1995) was a Canadian legal scholar, jurist, and human rights advocate. He is most famous as the author of the first draft of the Universal Declaration of Human Rights. Humphrey was born to Frank Humphrey and Nellie Peter on April 30, 1905, in Hampton, New Brunswick. His childhood was touched by tragedy as he lost both his parents to cancer and he also lost one of his arms in an accident while playing with fire. Humphrey attended a boarding school where he endured teasing from other students;
Document [3](Title: Drafting of the Universal Declaration of Human Rights) Drafting of the Universal Declaration of Human Rights The Universal Declaration of Human Rights was drafted from early 1947 to late 1948 by Drafting Committee the first United Nations Commission on Human Rights. Further discussion and amendments were made by the Commission on Human Rights, the Economic and Social Council and the General Assembly of the United Nations. Members of the Commission who contributed significantly to the creation of the Declaration included Canadian John Peters Humphrey of the United Nations Secretariat, Eleanor Roosevelt of the United States (who chaired the Drafting Committee), René Cassin of France, Charles Malik of Lebanon,
Document [4](Title: John Peters Humphrey) of the bench, which sits beside two tall stone plinths, one of which has several articles from the Universal Declaration of Human Rights carved into it in English, French and Maliseet. The memorial sits on the front lawn of the County Court House in the center of the town. John Peters Humphrey John Peters Humphrey, OC (April 30, 1905 – March 14, 1995) was a Canadian legal scholar, jurist, and human rights advocate. He is most famous as the author of the first draft of the Universal Declaration of Human Rights. Humphrey was born to Frank Humphrey and Nellie Peter
Document [5](Title: Cyrus Cylinder) The Cylinder's text has traditionally been seen by biblical scholars as corroborative evidence of Cyrus' policy of the repatriation of the Jewish people following their Babylonian captivity (an act that the Book of Ezra attributes to Cyrus), as the text refers to the restoration of cult sanctuaries and repatriation of deported peoples. This interpretation has been disputed, as the text identifies only Mesopotamian sanctuaries, and makes no mention of Jews, Jerusalem, or Judea. The Cylinder has also been referred to by Mohammad Reza Pahlavi, the last Shah of Iran as the first declaration of universal human rights, a view rejected by some historians as anachronistic and a misunderstanding of the Cylinder's generic nature as a typical statement made by a new monarch at the beginning of his reign. Neil MacGregor, Director of the British Museum, has stated that the cylinder was "the first attempt we know about running a society, a state with different nationalities and faiths—a new kind of statecraft." It was adopted as a national symbol of Iran by the Imperial State which put it on display in Tehran in 1971 to commemorate 2,500 year celebration of the Persian Empire. On October 14, the Mohammad Reza Shah's sister, Princess Ashraf Pahlavi, presented the United Nations Secretary General U Thant with a replica of the Cylinder. The princess asserted that "the heritage of Cyrus was the heritage of human understanding, tolerance, courage, compassion and, above all, human liberty".
Document [6](Title: Declaration of the Rights of Man and of the Citizen) version of the Declaration was discussed by the representatives on the basis of a 24 article draft proposed by , led by . The draft was later modified during the debates. A second and lengthier declaration, known as the Declaration of the Rights of Man and Citizen of 1793, was written in 1793 but never formally adopted. The concepts in the Declaration come from the philosophical and political duties of the Enlightenment, such as individualism, the social contract as theorized by the Genevan philosopher Rousseau, and the separation of powers espoused by the Baron de Montesquieu. As can be seen
Document [7](Title: Universal Declaration of Human Rights) committee included René Cassin of France, Charles Malik of Lebanon, and P. C. Chang of the Republic of China. Humphrey provided the initial draft that became the working text of the Commission. According to Allan Carlson, the Declaration's pro-family phrases were the result of the Christian Democratic movement's influence on Cassin and Malik. Once the Committee finished its work in May 1948, the draft was further discussed by the Commission on Human Rights, the Economic and Social Council, the Third Committee of the General Assembly before being put to vote in December 1948. During these discussions many amendments and propositions
Document [8](Title: Human rights in the United States) prisoners at Guantanamo Bay and black sites, and extrajudicial targeted killings (Disposition Matrix). Some observers give the U.S. high to fair marks on human rights, while others charge it with a persistent pattern of human rights violations. The first human rights organization in the Thirteen Colonies of British America, dedicated to the abolition of slavery, was formed by Anthony Benezet in 1775. A year later, the Declaration of Independence announced that the Thirteen Colonies regarded themselves as independent states, and no longer a part of the British Empire. The Declaration stated "that all men are created equal, that they are
Document [9](Title: John Peters Humphrey) Rights. After consulting with the executive group of the Commission, chaired by Eleanor Roosevelt, Professor Humphrey prepared the first preliminary draft of what was to become the Universal Declaration of Human Rights. On the night of December 10, 1948, the General Assembly unanimously adopted the Declaration, dubbed by Mrs. Eleanor Roosevelt as "the international Magna Carta of all humankind." Humphrey remained with the UN for 20 years. During this period, the oversaw the implementation of 67 international conventions and the constitutions of dozens of countries. He worked in areas including the freedom of the press, status of women, and racial
Document [10](Title: Universal Declaration of Human Rights) the Rights of the Child, the United Nations Convention Against Torture, and many more. The Declaration continues to be widely cited by governments, academics, advocates, and constitutional courts, and by individuals who appeal to its principles for the protection of their recognised human rights. The Universal Declaration has received praise from a number of notable people. The Lebanese philosopher and diplomat Charles Malik called it "an international document of the first order of importance", while Eleanor Roosevelt—first chairwoman of the Commission on Human Rights (CHR) that drafted the Declaration—stated that it "may well become the international Magna Carta of all

Question: who wrote the first declaration of human rights
Answer →"""

input_ids = tokenizer(few_shot_prompt, return_tensors="pt").input_ids.to('cuda')
outputs = model.generate(
    input_ids,
    max_new_tokens=32,
    eos_token_id=terminators,
    do_sample=False,
    temperature=None,
    top_p=None,
)

response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

  0%|          | 0/32 [00:00<?, ?it/s]

 the D the D the the D a the D of a D in a high a in the D the Declaration of the the


  0%|          | 0/32 [00:00<?, ?it/s]

 of a, of a
and of a
 the of a, a
the of a
of
the of a
of a
, of



In [5]:
clean_prompt = """Write a high-quality answer for the given question using only the provided search results (some of which might be irrelevant).

Document [1](Title: Nobel Prize in Physics) receive a diploma, a medal and a document confirming the prize amount. Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was
Document [2](Title: Norwegian Americans) science, Ernest Lawrence won the Nobel Prize in Physics in 1939. Lars Onsager won the 1968 Nobel Prize in Chemistry. Norman Borlaug, father of the Green Revolution, won the Nobel Peace Prize in 1970. Christian B. Anfinsen won the Nobel Prize for chemistry in 1972. Ivar Giaever won the Nobel Prize in Physics 1973. Carl Richard Hagen is noted for his work in physics. In engineering, Clayton Jacobson II is credited with the invention of the modern personal watercraft. Ole Singstad was a pioneer of underwater tunnels. Ole Evinrude invented the first outboard motor with practical commercial application, recognizable today
Document [3](Title: Nobel Prize in Physics) Nobel Prize in Physics The Nobel Prize in Physics () is a yearly award given by the Royal Swedish Academy of Sciences for those who have made the most outstanding contributions for mankind in the field of physics. It is one of the five Nobel Prizes established by the will of Alfred Nobel in 1895 and awarded since 1901; the others being the Nobel Prize in Chemistry, Nobel Prize in Literature, Nobel Peace Prize, and Nobel Prize in Physiology or Medicine. The first Nobel Prize in Physics was awarded to physicist Wilhelm Röntgen in recognition of the extraordinary services he
Document [4](Title: École normale supérieure (Paris)) was also awarded the Abel prize. In addition, eight "normaliens" have gone on to receive the Nobel Prize in Physics: Claude Cohen-Tannoudji, Pierre-Gilles de Gennes, Albert Fert, Alfred Kastler, Gabriel Lippmann, Louis Néel, Jean Baptiste Perrin and Serge Haroche, while other ENS physicists include such major figures as Paul Langevin, famous for developing Langevin dynamics and the Langevin equation. Alumnus Paul Sabatier won the Nobel Prize in Chemistry. A ranking of universities worldwide based on ratios of alumni to Nobel prize-winners published in 2016 by American scholars Stephen Hsu and Jonathan Wai placed ENS as the first university worldwide, far
Document [5](Title: List of Nobel laureates in Physics) The first Nobel Prize in Physics was awarded in 1901 to Wilhelm Conrad Röntgen, of Germany, who received 150,782 SEK, which is equal to 7,731,004 SEK in December 2007.  John Bardeen is the only laureate to win the prize twice—in 1956 and 1972. Maria Skłodowska-Curie also won two Nobel Prizes, for physics in 1903 and chemistry in 1911. William Lawrence Bragg was, until October 2014, the youngest ever Nobel laureate; he won the prize in 1915 at the age of 25. Two women have won the prize: Curie and Maria Goeppert-Mayer (1963). As of 2017, the prize has been awarded
Document [6](Title: Nobel Prize in Physics) rendered by the discovery of the remarkable rays (or x-rays). This award is administered by the Nobel Foundation and widely regarded as the most prestigious award that a scientist can receive in physics. It is presented in Stockholm at an annual ceremony on 10 December, the anniversary of Nobel's death. Through 2018, a total of 209 individuals have been awarded the prize. Only three women (1.4% of laureates) have won the Nobel Prize in Physics: Marie Curie in 1903, Maria Goeppert Mayer in 1963, and Donna Strickland in 2018. Alfred Nobel, in his last will and testament, stated that his
Document [7](Title: E. C. George Sudarshan) had developed the breakthrough. In 2007, Sudarshan told the "Hindustan Times", "The 2005 Nobel prize for Physics was awarded for my work, but I wasn't the one to get it. Each one of the discoveries that this Nobel was given for work based on my research." Sudarshan also commented on not being selected for the 1979 Nobel, "Steven Weinberg, Sheldon Glashow and Abdus Salam built on work I had done as a 26-year-old student. If you give a prize for a building, shouldn’t the fellow who built the first floor be given the prize before those who built the second
Document [8](Title: Svante Arrhenius) Wilhelm Ostwald, Theodore Richards) and to attempt to deny them to his enemies (Paul Ehrlich, Walther Nernst, Dmitri Mendeleev). In 1901 Arrhenius was elected to the Swedish Academy of Sciences, against strong opposition. In 1903 he became the first Swede to be awarded the Nobel Prize in chemistry. In 1905, upon the founding of the Nobel Institute for Physical Research at Stockholm, he was appointed rector of the institute, the position where he remained until retirement in 1927. He was elected a Foreign Member of the Royal Society (ForMemRS) in 1910. In 1911 he won the first Willard Gibbs Award.
Document [9](Title: Scientist) and pursued through a unique method, was essentially in place. Ramón y Cajal won the Nobel Prize in 1906 for his remarkable observations in neuroanatomy. Marie Curie became the first female to win the Nobel Prize and the first person to win it twice. Her efforts led to the development of nuclear energy and Radio therapy for the treatment of cancer. In 1922, she was appointed a member of the International Commission on Intellectual Co-operation by the Council of the League of Nations. She campaigned for scientist's right to patent their discoveries and inventions. She also campaigned for free access
Document [10](Title: Nobel Prize in Chemistry) on December 10, the anniversary of Nobel's death. The first Nobel Prize in Chemistry was awarded in 1901 to Jacobus Henricus van 't Hoff, of the Netherlands, "for his discovery of the laws of chemical dynamics and osmotic pressure in solutions." From 1901 to 2018, the award has been bestowed on a total of 180 individuals. Alfred Nobel stipulated in his last will and testament that his money be used to create a series of prizes for those who confer the "greatest benefit on mankind" in physics, chemistry, peace, physiology or medicine, and literature. Though Nobel wrote several wills during

Question: who got the first nobel prize in physics
Answer → Wilhelm Conrad Röntgen

Document [1](Title: Deadpool (video game)) 2015 in North America and Australia, and November 20, 2015 in the United Kingdom. It is unknown how Activision reacquired the rights to publish "Deadpool: The Video Game", but it is believed that the re-release was timed to help promote the then-upcoming Deadpool feature film, released roughly three months after the next-gen version of the game. High Moon Studios showed an early demo of the video game to journalists at Gamescom 2012, where the game earned a nomination for Best of Show. Several sites wrote positive impression of the demo that was shown at Gamescom and the Electronic Entertainment Expo
Document [2](Title: Deadpool (film)) footage was leaked online in July 2014, and was met with an overwhelmingly enthusiastic response. That September, Fox gave "Deadpool" a release date of February 12, 2016. Production was set to begin in March 2015, with Simon Kinberg joining as producer. Reynolds attributed Fox's green-lighting of the film entirely to the leak. He, Miller and the writers had previously discussed leaking the footage themselves, and Reynolds initially thought that Miller had done so. He later believed the leak came from someone at Fox. In exchange for being able to make the film the way they wanted, Fox gave the crew
Document [3](Title: Deadpool (film)) May of that year. Several vendors provided visual effects for the film, ranging from the addition of blood and gore to the creation of the CG character Colossus. "Deadpool" was released in the United States on February 12, 2016, after an unconventional marketing campaign. The film achieved both financial and critical success. It earned over $783 million against a $58 million budget, breaking numerous records: it became the highest-grossing R-rated film, the highest-grossing "X-Men" film, and the ninth-highest-grossing 2016 film. Critics praised Reynolds' performance, the film's style and faithfulness to the comics, and its action sequences. Some detractors criticized the
Document [4](Title: Paul Wernick) Tim Miller. The R-rated film was released on February 12, 2016. Its opening weekend was the highest grossing of all time for R-Rated films, and it went on to gross more than $780 million worldwide. The team's next project, sci-fi thriller "Life", was released in March 2017. The film was directed by Daniel Espinosa, produced by Skydance, and through Sony Pictures, with Ryan Reynolds, Jake Gyllenhaal, and Rebecca Ferguson starring. They wrote "Deadpool 2" in 2018, which like its predecessor was released to positive reviews. Wernick and Reese were announced as the co-screenwriters of the feature film adaptation of "Cowboy
Document [5](Title: Deadpool 2) Deadpool 2 is scheduled to be released in the United States on May 18, 2018.  A sequel, Deadpool 3, is in development.
Document [6](Title: Deadpool 2) film, explaining its general plot, was released at the end of March. "Forbes" contributor Scott Mendelson called it "pretty funny and mostly entertaining", but was disappointed in it being a "conventional" trailer compared to the more out-there videos previously released for the film. He explained that he thought the first film "had a winning lead character and fine character-centric jokes, but a pretty generic origin story plot that eventually became the thing it was critiquing", and was concerned that the sequel would turn out to be "a more standard 'superhero sequel' sell". Mendelson also noted the inclusion of T.J. Miller
Document [7](Title: Deadpool 2) at the convention moderated by Soni and featuring other cast members. Some of the scenes cut from the theatrical version of the film were debuted at the panel. At the end of September 2018, Fox announced that it would release an untitled "Deadpool" film in theaters on December 21. The studio suggested that press and fans "guess away" as to what this new film would be, but it was believed to be a re-cut version of "Deadpool 2" that would carry a PG-13 rating rather than being R-rated like the initial theatrical release. Reynolds hinted that the version of the
Document [8](Title: The New Mutants (film)) Bear would not be the main antagonist of the film, just that it was "very much inspired" by the run of the "New Mutants" comics in which Demon Bear was featured as the main villain. In January 2018, the film's release date was pushed back to February 22, 2019, which allowed it to avoid "Deadpool 2", which also had its release changed at that time to a date that would have had both films in theaters at the same time in certain markets, while also allowing time for the reshoots required to make the film more frightening. When asked about
Document [9](Title: Deadpool (film)) said that the company would be willing to make future R-rated Marvel films like "Deadpool", potentially under a "Marvel-R" brand, "as long as we let the audiences know what's coming". Before "Deadpool"s release, Fox green-lit a sequel with Reese and Wernick returning to write the screenplay. The involvement of Reynolds and Tim Miller was confirmed at the 2016 CinemaCon in April, but at the end of October, Miller left the film over "mutual creative differences" with Reynolds. The next month, David Leitch signed on to replace Miller for the sequel. Leitch first made a short film, "No Good Deed", which
Document [10](Title: Deadpool (film)) Screen Rant called it possibly "the best film marketing campaign in the history of cinema". HostGator's Jeremy Jensen attributed the campaign's success to Reynolds, and to Fox for embracing the film's R rating. "Deadpool"s world premiere was held at the Grand Rex in Paris on February 8, 2016, before its initial theatrical release in Hong Kong the next day. This was followed by releases in 49 other markets over the next few days, including the United States on February 12. The movie was released in several formats, including IMAX, DLP, premium large formats, and D-Box. Kinberg explained that unlike the

Question: when is the next deadpool movie being released
Answer → May 18, 2018

Document [1](Title: Geography of Nigeria) south atlantic ocean, locally known as the south western wind, or by its main name, The Tropical Maritime (MT) airmass. These two major wind systems in Nigeria are known as the trade winds. The tropical maritime airmass (MT) is responsible for Nigeria's rainy season. This wind (the tropical maritime airmass) invades the country from February in the southern part of Nigeria while it takes longer for the wind to fully cover the whole of the country, reaching the northern part of Nigeria in June. Its invasion is as a result of the northward retreat, of the tropical continental airmass (CT)
Document [2](Title: Geography of Nigeria) of low pressure to develop over west Africa and Nigeria (between March to May). The Tropical continental airmass (CT) from the Sahara Desert in the northern part of West Africa, is weakened due to the overheating of the land surface in west Africa and Nigeria at this time. The Tropical continental airmass (CT) begins to retreat northwards to the Sahara Desert due to massive heating of the land which transfers heat in the form of convection into the Tropical continental airmass (CT) which constitutes the main layer of air above the land. This transfer of heat in the Tropical continental
Document [3](Title: Geography of Nigeria) airmass (CT) in turn, causes the wind to expand and become lighter as this is the normal behaviour for winds moving above intensely heated grounds. The Tropical continental airmass (CT) loses its strength as a major airmass in the region of west Africa and over Nigeria at this time (around February in the southern part of Nigeria to June in northern Nigeria) and begins to retreat coupled with the rising of air in form of convection within this airmass (Tropical continental airmass (CT)), further weakening the dominance of the wind over west Africa and Nigeria. The Tropical continental airmass (CT)
Document [4](Title: Geography of Nigeria) northern end is south of the 15 degrees line at about 14 degrees. Nigeria's location in the wetter part of the easterly waves south of the 15 degree line creates wetter climatic conditions for Nigeria especially during the monsoons. The Tropical Continental Airmass (CT) locally known as the harmattan, is a wind originating from North Africa which crosses the Sahara Desert into west Africa to Nigeria. This airmass dominates Nigeria's climate during the dry season from December to March. The Tropical continental airmass is dusty and creates a haze within the atmosphere of west Africa and Nigeria when it predominates.
Document [5](Title: Geography of Nigeria) With the Intertropical Convergence Zone (ITCZ) swinging northward over West Africa from the Southern Hemisphere in April, heavy showers coming from pre-monsoonal convective clouds mainly in the form of squall lines also known as the north easterlies formed mainly as a result of the interactions of the two dominant airmasses in Nigeria known as the Maritime tropical(south westerlies) and the Continental tropical(north easterlies), begins in central Nigeria while the Monsoons from the south atlantic ocean arrives in central Nigeria in July bringing with it high humidity, heavy cloud cover and heavy rainfall which can be daily occurrence lasting till September when the monsoons gradually begin retreating southward to the southern part of Nigeria.
Document [6](Title: Awka) dry winds from across the Sahara desert. The monsoon winds from the Atlantic creates six months of heavy tropical rains, which occur between April and July, followed by a short dry period in August lasting two to three weeks with the rain resuming in September and October. This is followed by five months of dryness (November - March) marked by a Harmattan wind, also known as Ugulu in Igbo, which is a particularly dry and dusty wind which enters Nigeria in late December or in the early part of January and is characterized by a grey haze limiting visibility and
Document [7](Title: Geography of Nigeria) experiences inward blowing winds because winds are moving air blowing outwards from regions of high pressure to regions of low pressure. The Tropical Maritime Airmass is a warm humid and unstable trade wind due to its warmth. Convectional currents are easily set up within the airmass whenever there is little instability in the airmass as a result of a slight to a very high orographic uplift in mountainous regions like the obudu plateau or the heating of the land which can trigger the formation of cumulonimbus cloud leading to thunderstorms within the airmass. During the dominance of the Tropical Maritime
Document [8](Title: Geography of Nigeria) finally retreats from most part of Nigeria, and the West African atmosphere around April to May, leaving an empty atmosphere over Nigeria. The sun's rays enters into the atmosphere of Nigeria more intense than it does during the presence of the Tropical continental airmass, which contained dust (in form of haze) that reduced the intensity of the sun. The overheating of the west Africa land mass and Nigeria in particular creates a low pressure region over west Africa and Nigeria. This low pressure zone attracts the Tropical Maritime Airmass (MT) from the south Atlantic Ocean since areas of low pressures
Document [9](Title: Western Asia) two wind phenomena in Western Asia: the "sharqi" and the "shamal". The "sharqi" (or "sharki") is a wind that comes from the south and southeast. It is seasonal, lasting from April to early June, and comes again between late September and November. The winds are dry and dusty, with occasional gusts up to 80 kilometres per hour (50 miles per hour) and often kick up violent sand and dust storms that can carry sand a few thousand meters high, and can close down airports for short periods of time. These winds can last for a full day at the beginning
Document [10](Title: Onikwu) The dry season is accompanied by a dust laden airmass from the Sahara Desert, locally known as Harmattan, or by its main name, The Tropical Continental (CT) airmass, while the rainy season is heavily influenced by an airmass originating from the south Atlantic Ocean, locally known as the south west wind, or by its main name, The Tropical Maritime (MT) airmass. These two major wind systems in Nigeria are known as the trade winds. The region Onikwu/Ndoni is flood prone communities, this is because the inland part of Rivers state consists of tropical rainforest; towards the coast the typical Niger

Question: the south west wind blows across nigeria between
Answer → till September

Document [1](Title: Order of War) Order of War Order of War is a World War II Strategy video game developed by Wargaming.net and published by Square Enix. It was released on September 22, 2009. In the summer of 1944, World War II entered its end-game when the Allies launched two major operations that would drive back German forces on both the Western and Eastern Fronts at once. Order of War contains two campaigns, one American and one German. Order of War allows players to lead American forces to victory in the west and also places players in command of the German forces charged with repelling
Document [2](Title: Dark Ages (1999 video game)) boost stats so they can learn skills with the fewest points actually spent possible, enabling the excess points to be spent in CON or WIS, allowing the character to enjoy a greatly improved HP or MP pool and regeneration while leveling. HP and MP are the standards hit point and magic points. HP is reduced from receiving damage in combat from enemies, and if it is reduced to 0 results in the death of the character except under special circumstances. If a character is a member of a party, instead of immediate death the character is in a swooning state
Document [3](Title: Hume's principle) Hume's principle Hume's principle or HP—the terms were coined by George Boolos—says that the number of "F"s is equal to the number of "G"s if and only if there is a one-to-one correspondence (a bijection) between the "F"s and the "G"s. HP can be stated formally in systems of second-order logic. Hume's principle is named for the Scottish philosopher David Hume. HP plays a central role in Gottlob Frege's philosophy of mathematics. Frege shows that HP and suitable definitions of arithmetical notions entail all axioms of what we now call second-order arithmetic. This result is known as Frege's theorem, which
Document [4](Title: Order of War) the Red Army’s offensive in the east. Order of War features detailed graphics and a historically accurate storyline. The main focus of the game that sets it apart from other strategy offers is the sweeping scale of battlefield vistas featuring company level engagements. Players take command of multiple unit types, including ground troops, artillery, tanks and aircraft in strategic assaults based on key missions and operations from 1944. Order of War is a Real Time Strategy (RTS) game without requiring the player to manage traditional base building or resource collection activities. Much like the Relic Entertainment's "" series or the
Document [5](Title: Health (gaming)) Health or vitality is an attribute assigned to entities, such as the player character, enemies and objects within a role-playing or video game, that indicates its state in combat. Health is usually measured in hit points or health points, shortened to HP.  When the HP of a player character reaches zero, the player may lose a life or their character might become incapacitated or die. When the HP of an enemy reaches zero, it may be defeated or die and the player is usually rewarded in some way.
Document [6](Title: War and Peace (game)) Spain and Russia where it was harder for armies to live off the land; all units in their home country get a -1 DRM whereas unsupplied units get a +1 DRM). If more than one strength point is to be lost then one of them must be cavalry if there is one in the hex. Neutral countries are immune from attrition, enabling a defeated major power to rebuild their forces until they rejoin the war. Diplomacy takes place in the alliance phase. This system is used to allow nations to ally with one another. Alliances essentially allow you to acquire
Document [7](Title: Order of battle) the later transformation of its meaning during the European period of Early Modern warfare the order of battle came to mean the order in which the units manoeuvered or deployed onto the battlefield to form battle-lines, with the positioning on the right considered the place of greatest honour. This need to reflect the unit seniority led to the keeping of military staff records, in tabular form reflecting the compilation of units an army, their commanders, equipment, and locations on the battlefield. During the Napoleonic wars the meaning of the order of battle changed yet again to reflect the changes in
Document [8](Title: Warhammer: Mark of Chaos) cause them to lose defense and speed when sufficiently drained. The troops present in the game are presented as "units", which is anything from 1 to 96 individual "models", with the specific number depending on the type of unit. The control system is similar to the Total War and in that orders are issued to units in their entirety, as opposed to being issued to individuals. As well as standard orders you are also able to arrange your units into a number of formations, with each formation conferring advantages against specific kinds of attacks. Units will also gain experience over
Document [9](Title: Horsepower-hour) Horsepower-hour A horsepower-hour (hph or hp⋅h) is an outdated unit of energy, not used in the SI system of units. The unit represents an amount of work a horse is supposed capable of delivering during an hour (1 (one) horsepower integrated over a time interval of an hour). Based on differences in the definition of what constitutes the 'power of a horse', a horsepower-hour differs slightly from the German "Pferdestärkenstunde" (PSh): The horsepower-hour is still used in the railroad industry when sharing motive power (locomotives.) For example, if Railroad A borrows a 2,500 horsepower locomotive from Railroad B and operates
Document [10](Title: Dragon Quest (video game)) If his HP falls to zero, he dies and is taken back to King Lorik to be resurrected, and loses half his gold "as punishment." If the hero succeeds in defeating an enemy, he gains experience points and gold; if he gains enough experience points, his experience level increases, giving him greater strength, agility, speed, and the ability to use magic spells. Every time a spell is used, the hero's MP decreases, with different spells costing different amounts of MP. Both HP and MP can be restored by resting at an inn. Additionally, a non-player character can replenish the hero's

Question: what does hp mean in war and order
Answer →"""

clean_tokens = tokenizer(clean_prompt, return_tensors="pt").input_ids.to('cuda')

activations_store = torch.zeros((model.cfg.n_layers, model.cfg.d_model), device=model.cfg.device)


def activations_store_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
):
    activations_store[hook.layer()] = resid_pre[:, -1, :]
    # print(hook.layer())
    

# for layer in range(model.cfg.n_layers):
temp_hook_fn = partial(activations_store_hook)
clean_logits = model.run_with_hooks(
    clean_tokens,
    fwd_hooks=[
    (utils.get_act_name("resid_pre", layer), temp_hook_fn) for layer in range(model.cfg.n_layers)
])
# clean_logits, clean_cache = model.run_with_cache(clean_tokens)
# tokenizer.decode(clean_logits[0, -1, :].argmax())

In [6]:
len(clean_tokens[0])

5615

In [7]:
activations_store.shape

torch.Size([32, 4096])

In [8]:
test_prompt = """Write a high-quality answer for the given question using only the provided search results (some of which might be irrelevant).

Document [1](Title: Drafting of the Universal Declaration of Human Rights) P. C. Chang of Republic of China (Taiwan), and Hansa Mehta of India among others. While not a member of the drafting committee, the French philosopher Jacques Maritain was influential in the lead up to the drafting of the Universal Declaration, advocacy for it within UNESCO in 1947-8, and in its subsequent advancement. The Drafting Committee included John Peters Humphrey was newly appointed as Director of the Division of Human Rights within the United Nations Secretariat. In this role, he produced the first draft of a list of rights that were to form the basis of the Declaration. The underlying
Document [2](Title: John Peters Humphrey) John Peters Humphrey John Peters Humphrey, OC (April 30, 1905 – March 14, 1995) was a Canadian legal scholar, jurist, and human rights advocate. He is most famous as the author of the first draft of the Universal Declaration of Human Rights. Humphrey was born to Frank Humphrey and Nellie Peter on April 30, 1905, in Hampton, New Brunswick. His childhood was touched by tragedy as he lost both his parents to cancer and he also lost one of his arms in an accident while playing with fire. Humphrey attended a boarding school where he endured teasing from other students;
Document [3](Title: Drafting of the Universal Declaration of Human Rights) Drafting of the Universal Declaration of Human Rights The Universal Declaration of Human Rights was drafted from early 1947 to late 1948 by Drafting Committee the first United Nations Commission on Human Rights. Further discussion and amendments were made by the Commission on Human Rights, the Economic and Social Council and the General Assembly of the United Nations. Members of the Commission who contributed significantly to the creation of the Declaration included Canadian John Peters Humphrey of the United Nations Secretariat, Eleanor Roosevelt of the United States (who chaired the Drafting Committee), René Cassin of France, Charles Malik of Lebanon,
Document [4](Title: John Peters Humphrey) of the bench, which sits beside two tall stone plinths, one of which has several articles from the Universal Declaration of Human Rights carved into it in English, French and Maliseet. The memorial sits on the front lawn of the County Court House in the center of the town. John Peters Humphrey John Peters Humphrey, OC (April 30, 1905 – March 14, 1995) was a Canadian legal scholar, jurist, and human rights advocate. He is most famous as the author of the first draft of the Universal Declaration of Human Rights. Humphrey was born to Frank Humphrey and Nellie Peter
Document [5](Title: Cyrus Cylinder) The Cylinder's text has traditionally been seen by biblical scholars as corroborative evidence of Cyrus' policy of the repatriation of the Jewish people following their Babylonian captivity (an act that the Book of Ezra attributes to Cyrus), as the text refers to the restoration of cult sanctuaries and repatriation of deported peoples. This interpretation has been disputed, as the text identifies only Mesopotamian sanctuaries, and makes no mention of Jews, Jerusalem, or Judea. The Cylinder has also been referred to by Mohammad Reza Pahlavi, the last Shah of Iran as the first declaration of universal human rights, a view rejected by some historians as anachronistic and a misunderstanding of the Cylinder's generic nature as a typical statement made by a new monarch at the beginning of his reign. Neil MacGregor, Director of the British Museum, has stated that the cylinder was "the first attempt we know about running a society, a state with different nationalities and faiths—a new kind of statecraft." It was adopted as a national symbol of Iran by the Imperial State which put it on display in Tehran in 1971 to commemorate 2,500 year celebration of the Persian Empire. On October 14, the Mohammad Reza Shah's sister, Princess Ashraf Pahlavi, presented the United Nations Secretary General U Thant with a replica of the Cylinder. The princess asserted that "the heritage of Cyrus was the heritage of human understanding, tolerance, courage, compassion and, above all, human liberty".
Document [6](Title: Declaration of the Rights of Man and of the Citizen) version of the Declaration was discussed by the representatives on the basis of a 24 article draft proposed by , led by . The draft was later modified during the debates. A second and lengthier declaration, known as the Declaration of the Rights of Man and Citizen of 1793, was written in 1793 but never formally adopted. The concepts in the Declaration come from the philosophical and political duties of the Enlightenment, such as individualism, the social contract as theorized by the Genevan philosopher Rousseau, and the separation of powers espoused by the Baron de Montesquieu. As can be seen
Document [7](Title: Universal Declaration of Human Rights) committee included René Cassin of France, Charles Malik of Lebanon, and P. C. Chang of the Republic of China. Humphrey provided the initial draft that became the working text of the Commission. According to Allan Carlson, the Declaration's pro-family phrases were the result of the Christian Democratic movement's influence on Cassin and Malik. Once the Committee finished its work in May 1948, the draft was further discussed by the Commission on Human Rights, the Economic and Social Council, the Third Committee of the General Assembly before being put to vote in December 1948. During these discussions many amendments and propositions
Document [8](Title: Human rights in the United States) prisoners at Guantanamo Bay and black sites, and extrajudicial targeted killings (Disposition Matrix). Some observers give the U.S. high to fair marks on human rights, while others charge it with a persistent pattern of human rights violations. The first human rights organization in the Thirteen Colonies of British America, dedicated to the abolition of slavery, was formed by Anthony Benezet in 1775. A year later, the Declaration of Independence announced that the Thirteen Colonies regarded themselves as independent states, and no longer a part of the British Empire. The Declaration stated "that all men are created equal, that they are
Document [9](Title: John Peters Humphrey) Rights. After consulting with the executive group of the Commission, chaired by Eleanor Roosevelt, Professor Humphrey prepared the first preliminary draft of what was to become the Universal Declaration of Human Rights. On the night of December 10, 1948, the General Assembly unanimously adopted the Declaration, dubbed by Mrs. Eleanor Roosevelt as "the international Magna Carta of all humankind." Humphrey remained with the UN for 20 years. During this period, the oversaw the implementation of 67 international conventions and the constitutions of dozens of countries. He worked in areas including the freedom of the press, status of women, and racial
Document [10](Title: Universal Declaration of Human Rights) the Rights of the Child, the United Nations Convention Against Torture, and many more. The Declaration continues to be widely cited by governments, academics, advocates, and constitutional courts, and by individuals who appeal to its principles for the protection of their recognised human rights. The Universal Declaration has received praise from a number of notable people. The Lebanese philosopher and diplomat Charles Malik called it "an international document of the first order of importance", while Eleanor Roosevelt—first chairwoman of the Commission on Human Rights (CHR) that drafted the Declaration—stated that it "may well become the international Magna Carta of all

Question: who wrote the first declaration of human rights
Answer →"""

def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    # Each HookPoint has a name attribute giving the name of the hook.
    resid_pre[:, position, :] = activations_store[hook.layer()]
    return resid_pre

def logits_to_logit_diff(logits, correct_answers=[' Cyrus', ' Cylinder'], incorrect_answer=" the"):
    # model.to_single_token maps a string value of a single token to the token index for that token
    # If the string is not a single token, it raises an error.
    diff_max = -1e9
    for correct_answer in correct_answers:
        correct_index = model.to_single_token(correct_answer)
        incorrect_index = model.to_single_token(incorrect_answer)
        diff = logits[0, -1, correct_index] - logits[0, -1, incorrect_index]
        diff_max = max(diff, diff_max)
    return diff_max

test_tokens = tokenizer(test_prompt, return_tensors="pt").input_ids.to('cuda')
num_positions = len(test_tokens[0])
ioi_patching_result = torch.zeros((model.cfg.n_layers, num_positions), device=model.cfg.device)


for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    position = test_tokens.shape[1] - 1
    # Use functools.partial to create a temporary hook function with the position fixed
    temp_hook_fn = partial(residual_stream_patching_hook, position=position)
    # Run the model with the patching hook
    patched_logits = model.run_with_hooks(test_tokens, fwd_hooks=[
        (utils.get_act_name("resid_pre", layer), temp_hook_fn)
    ])
    # Calculate the logit difference
    patched_logit_diff = logits_to_logit_diff(patched_logits).detach()
    print('layer ', layer, 'decoded', repr(tokenizer.decode(patched_logits[0, -1, :].argmax())), patched_logit_diff)

  0%|          | 0/32 [00:00<?, ?it/s]

layer  0 decoded '\xa0' tensor(-15.0625, device='cuda:0', dtype=torch.bfloat16)
layer  1 decoded '\xa0' tensor(-15.1875, device='cuda:0', dtype=torch.bfloat16)
layer  2 decoded '\xa0' tensor(-15.0625, device='cuda:0', dtype=torch.bfloat16)
layer  3 decoded ' the' tensor(-14.6250, device='cuda:0', dtype=torch.bfloat16)
layer  4 decoded ' the' tensor(-14.2500, device='cuda:0', dtype=torch.bfloat16)
layer  5 decoded ' the' tensor(-14.1250, device='cuda:0', dtype=torch.bfloat16)
layer  6 decoded ' and' tensor(-13.1250, device='cuda:0', dtype=torch.bfloat16)
layer  7 decoded ' and' tensor(-14.6250, device='cuda:0', dtype=torch.bfloat16)
layer  8 decoded ' and' tensor(-15.6250, device='cuda:0', dtype=torch.bfloat16)
layer  9 decoded ' and' tensor(-15.6250, device='cuda:0', dtype=torch.bfloat16)
layer  10 decoded ' and' tensor(-15.3750, device='cuda:0', dtype=torch.bfloat16)
layer  11 decoded ' in' tensor(-14.8750, device='cuda:0', dtype=torch.bfloat16)
layer  12 decoded ' and' tensor(-16.625

In [9]:
tokenizer.encode(' Cyrus')

[128000, 71924]

In [10]:
tokenizer.encode(' Cylinder')

[128000, 87762]

In [11]:
## TODO: 矢印でやる
## TODO: zero-shot, few-shotのbaseline, task vector(few-shotのtask vector, few-shot-CoTのtask vector)
## TODO: key-value task